In [ ]:
import ast
import pycountry

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns
import plotly.graph_objects as go

from matplotlib import colors
from pycountry_convert import country_alpha2_to_continent_code
from iso_language_codes import *

import warnings
warnings.filterwarnings('ignore')

sns.set_style('whitegrid')
sns.set(font_scale=1.25)

In [ ]:
# wczytanie danych
df = pd.read_csv('data/movies_metadata.csv')
print(df.shape)
df.head()

In [ ]:
df.info()

Tylko w kolumnach `belongs_to_collection`, `homepage` oraz `tagline` występują brakujące wartości.

## Kraj produkcji i język

Domyślamy się, że większość filmów jest w języku angielskim, jednakże, filmy te mogłby być kręcone w różnych miejscach na świecie.

Interesujące byłoby sprawdzenie, które kraje są najpopularniejszymi miejscami kręcenia filmów oraz w jakich językach się najczęściej mówi.

In [ ]:
# przykładowy kraj produkcji

df['production_countries'].loc[0]

String z listą z JSONem

In [ ]:
# ile rekordów nie ma kraju produkcji

df['production_countries'].isna().sum()

In [ ]:
# uzupełniamy brakujące wartości pustą listą, używamy ast.literval_eval bo lista jest w stringu
df['production_countries'] = df['production_countries'].fillna('[]').apply(ast.literal_eval)

# wyciągamy wartość 'name' z kolumny z krajem produkcji
df['production_countries'] = df['production_countries'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [ ]:
# przykład

df['production_countries'].loc[0]

In [ ]:
# istnieją także przypadki w których film jest tworzony w kilku krajach

df['production_countries'].loc[6]

In [ ]:
# przekształć każdy element listy w wiersz, replikując wartości indeksów

prod_count = df['production_countries'].explode()
prod_count = pd.DataFrame({'kraj': prod_count.value_counts().index,
                           'liczba filmów': prod_count.value_counts().values}).reset_index(drop=True)

# zamiana nieistniejących już krajów; zamiana nazw nieczytelnych dla formatu alpha2
prod_count.loc[prod_count.kraj == "United States of America", "kraj"] = "USA"
prod_count.loc[prod_count.kraj == "Soviet Union", "kraj"] = "Russia"
prod_count.loc[prod_count.kraj == "East Germany", "kraj"] = "Germany"
prod_count.loc[prod_count.kraj == "Cote D'Ivoire", "kraj"] = "Côte d'Ivoire"
prod_count.loc[prod_count.kraj == "French Southern Territories", "kraj"] = "France"
prod_count.loc[prod_count.kraj == "United States Minor Outlying Islands", "kraj"] = "USA"
prod_count.loc[prod_count.kraj == "Libyan Arab Jamahiriya", "kraj"] = "Libya"
prod_count.loc[prod_count.kraj == "Serbia and Montenegro", "kraj"] = "Serbia"
prod_count.loc[prod_count.kraj == "Yugoslavia", "kraj"] = "Serbia"
prod_count.loc[prod_count.kraj == "Netherlands Antilles", "kraj"] = "Netherlands"
prod_count.loc[prod_count.kraj == "Czechoslovakia", "kraj"] = "Czech Republic"
prod_count.loc[prod_count.kraj == "Kyrgyz Republic", "kraj"] = "Kyrgyzstan"
prod_count.loc[prod_count.kraj == "Macedonia", "kraj"] = "North Macedonia"
prod_count.loc[prod_count.kraj == "Palestinian Territory", "kraj"] = "Palestine"

# dodawanie duplikatów (Russia + Soviet Union po zamienieniu)
prod_count = prod_count.groupby(['kraj'], as_index=False).agg({'liczba filmów': 'sum'})

In [ ]:
# tworzenie kodu notacji alpha2 z nazwy państwa
input_countries = prod_count['kraj']
countries = {}
for country in pycountry.countries:
    countries[country.name] = country.alpha_2
codes = [countries.get(country, 'Unknown code') for country in input_countries]
prod_count['code'] = codes

# zamiana alpha2 na kontynent dla tych krajów, które mają alpha2
with_alpha2 = prod_count[~prod_count['code'].isin(['AQ', 'Unknown code'])]
with_alpha2['continent'] = [country_alpha2_to_continent_code(code) for code in with_alpha2['code']]

# zamiana alpha2 na kontynent dla tych krajów, które go nie mają
without_alpha2 = prod_count[prod_count['code'].isin(['AQ', 'Unknown code'])]

without_alpha2.loc[without_alpha2.kraj.isin(['Antarctica']), 'continent'] = 'AQ'
without_alpha2.loc[without_alpha2.kraj.isin(['Bolivia', 'Venezuela']), 'continent'] = 'SA'
without_alpha2.loc[without_alpha2.kraj.isin(['Czech Republic', 'Moldova', '']), 'continent'] = 'EU'
without_alpha2.loc[without_alpha2.kraj.isin(['Iran', 'North Korea', 'Palestine', 'Russia', 'South Korea', 'Taiwan', 'Vietnam']), 'continent'] = 'AS'
without_alpha2.loc[without_alpha2.kraj.isin(['Tanzania']), 'continent'] = 'AF'
without_alpha2.loc[without_alpha2.kraj.isin(['USA']), 'continent'] = 'NA'

# połączenie
prod_count = pd.concat([with_alpha2, without_alpha2]).sort_index()

In [ ]:
# mapowanie skrótu na pełną nazwę

continents = {
    'AS': 'Asia',
    'NA': 'North America',
    'SA': 'South America', 
    'OC': 'Australia',
    'AF': 'Africa',
    'EU': 'Europe',
    'AQ': 'Antartica'
}

prod_count['continent'] = prod_count['continent'].map(continents)

In [ ]:
print(prod_count.to_string(index=False))

In [ ]:
fig = px.sunburst(prod_count, path=['continent', 'kraj'], values='liczba filmów',
                  color='continent',
                  color_continuous_scale='RdBu',
                  labels={"parent": "parent"})

fig.update_traces(
        go.Sunburst(hovertemplate='<b>%{label} </b> <br>Liczba filmów: %{value}'),
        insidetextorientation='radial',       
    )

fig.show()

In [ ]:
# błędy w języku
df = df.drop(df[df.original_language.isin(['104.0', '68.0', '82.0', 'xx'])].index)
df = df.drop(df[df.original_language.isna()].index)

# liczba unikalnych języków
uni_lang = pd.DataFrame(df['original_language'].drop_duplicates().unique())
uni_lang[uni_lang == 'cn'] = 'zh'
print('\nJest {} różnych języków.'.format(len(uni_lang)))

In [ ]:
# dataframe z zagregowanymi wartoścami
lang_df = df['original_language'].value_counts().rename_axis('lang_code').reset_index(name='counts')

# mandarin to chinese
lang_df.loc[lang_df.lang_code == "cn", "lang_code"] = "zh"
lang_df = lang_df.groupby(['lang_code'], as_index=False).agg({'counts': 'sum'})

# zamiana skrótu na język
lang_df['language'] = lang_df['lang_code'].apply(lambda x: language(x)['Name'])
lang_df = lang_df.sort_values(by=['counts'], ascending=False)

In [ ]:
plt.figure(figsize=(12,5))
s = sns.barplot(x='language', y='counts', data=lang_df.iloc[1:11])
s.set(xlabel='Język', ylabel='Liczba filmów')
plt.show()